# libraries imports

In [1]:
from datasets import load_dataset
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas

2025-04-16 16:07:18.272778: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-16 16:07:18.272871: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-16 16:07:18.272925: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-16 16:07:19.655488: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# preposcessing the images 

In [2]:
# Load Fashion MNIST from Hugging Face
dataset = load_dataset("fashion_mnist")

# Extract images and labels
X_train = np.array(dataset['train']['image'])
y_train = np.array(dataset['train']['label'])
X_test = np.array(dataset['test']['image'])
y_test = np.array(dataset['test']['label'])

# Normalize pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

# Reshape to (28, 28, 1) for CNN
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# Split validation from train
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42)

# model architecture and compilinh 

In [3]:
# Define a simple CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')  # 10 clothing categories
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_val, y_val),
    verbose=1
)

Epoch 1/10
1688/1688 [==============================] - 37s 21ms/step - loss: 0.4791 - accuracy: 0.8278 - val_loss: 0.3470 - val_accuracy: 0.8735
Epoch 2/10
1688/1688 [==============================] - 35s 21ms/step - loss: 0.3189 - accuracy: 0.8846 - val_loss: 0.2845 - val_accuracy: 0.8990
Epoch 3/10
1688/1688 [==============================] - 34s 20ms/step - loss: 0.2731 - accuracy: 0.9001 - val_loss: 0.2679 - val_accuracy: 0.9008
Epoch 4/10
1688/1688 [==============================] - 34s 20ms/step - loss: 0.2420 - accuracy: 0.9116 - val_loss: 0.2465 - val_accuracy: 0.9082
Epoch 5/10
1688/1688 [==============================] - 34s 20ms/step - loss: 0.2164 - accuracy: 0.9199 - val_loss: 0.2492 - val_accuracy: 0.9058
Epoch 6/10
1688/1688 [==============================] - 34s 20ms/step - loss: 0.1935 - accuracy: 0.9278 - val_loss: 0.2492 - val_accuracy: 0.9093
Epoch 7/10
1688/1688 [==============================] - 34s 20ms/step - loss: 0.1742 - accuracy: 0.9356 - val_loss: 0.2311 -

# model evaluation 

In [4]:
# Evaluate the trained model on the test set
# This gives a true performance estimate on unseen data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)

# Print the final test accuracy in a readable format
print(f"\nTest Accuracy: {test_accuracy:.4f}")

# Save the trained model to a file inside the 'models' folder
# This allows us to reuse it later without retraining
model.save("models/stage1_fashion_cnn.h5")

# Confirm that the model has been saved successfully
print("Model saved to models/stage1_fashion_cnn.h5")

313/313 - 2s - loss: 0.2986 - accuracy: 0.9038 - 2s/epoch - 7ms/step

Test Accuracy: 0.9038
Model saved to models/stage1_fashion_cnn.h5


/opt/jupyterhub/pyvenv/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
